In [26]:
from ipynb.fs.full.hb2 import Hummingbird2
from ipynb.fs.full.dh import DiffieHellman
from ipynb.fs.full.keygen import Person
from ipynb.fs.full.cha import chachaE
from ipynb.fs.full.aes import aesE
from ipynb.fs.full.xtea import *
from ipynb.fs.full.pypresent import Present
import codecs
from time import process_time

# Run the autokey encryption method running on the IoT device
def autokeyEncryption(plaintext, iterations):
    secret = -85
    res = []
    start = process_time()

    for i in range(iterations):
        for idx in range(len(plaintext)):
            b = (secret ^ plaintext[idx])
            secret = plaintext[idx]
            res.append(b)

    end = process_time()
    return plaintext, (end - start)

def _textToByteArray(text):
    return bytearray(text, 'utf-8')

# HB2 Implementation used from https://github.com/bozhu/Hummingbird2-Python/blob/master/hb2.py
# Full credits to author for implementation of the HB2 Algorithm
def hb2Encryption(plaintext, iterations):
    key = (0x2301, 0x6745, 0xAB89, 0xEFCD, 0xDCFE, 0x98BA, 0x5476, 0x1032)
    iv  = (0x3412, 0x7856, 0xBC9A, 0xF0DE)
    hb2 = Hummingbird2(key, iv)
    
    start = process_time()
    for i in range(iterations):
        for p in plaintext:
            ciphertext = hb2.enc(p)
    end = process_time()
    return ciphertext, (end - start)
 
def key(plaintext, iterations):
    alice = Person()
    bob = Person()
    start = process_time()
    for i in range(iterations):
        key = bob.keyGen(alice.public_key)
    end = process_time()
    return key, (end - start)

def ECDHencrypt(plaintext, iterations):
    alice = DiffieHellman()
    bob = DiffieHellman()
    start = process_time()
    for i in range(iterations):
        encrypted_message = bob.encrypt(alice.public_key, plaintext)
    end = process_time()
#     decrypted_message = alice.decrypt(bob.public_key, encrypted_message, bob.IV)
#     print(decrypted_message)
    return encrypted_message, (end - start)

# def present(plaintext, iterations):
#     key = codecs.decode(b"0123456789abcdef0123456789abcdef", 'hex')
#      "0123456789abcdef0123456789abcdef".decode('hex')
#     cipher = Present(key)
#     start = process_time()
#     for i in range(iterations):
#         encrypted_message = cipher.encrypt(plaintext)
#     end = process_time()
#     return encrypted_message, (end - start)


def aes(plaintext, iterations):
    start = process_time()
    for i in range(iterations):
        encrypted_message = aesE(plaintext)
    end = process_time()
    return encrypted_message, (end - start)

def chacha(plaintext, iterations):
    start = process_time()
    for i in range(iterations):
        encrypted_message = chachaE(plaintext)
    end = process_time()
    return encrypted_message, (end - start)
        
def xsmth(plaintext, iterations):
    key = b" "*16
    x = new(key, mode=MODE_OFB, IV=b"12345678")
    start = process_time()
    for i in range(iterations):
            c = x.encrypt(plaintext)
    end = process_time()
    return c, (end - start)

iterations = 1000
nbbytes=5;
plaintext = '{\"system\":{\"set_relay_state\":{\"state\":1}}}'
plaintext = _textToByteArray(plaintext)



print ('Calling autokey encryption')
ciphertext, elapsed_ak = autokeyEncryption(plaintext, iterations)
print ('Took ' + str(elapsed_ak) + ' on average')
print ('Cipher text: ' + str(ciphertext))
text = "{\"system\":{\"set_relay_state\":{\"state\":1}}}"

print ('Calling ecdh encryption')
ciphertext, elapsed_ecdh = ECDHencrypt(text, iterations)
print ('Took ' + str(elapsed_ecdh) + ' on average')
print ('Cipher text: ' + str(ciphertext))

text = b"{\"system\":{\"set_relay_state\":{\"state\":1}}}"

print ('Calling chacha encryption')
ciphertext, elapsed_chacha = chacha(text, iterations)
print ('Took ' + str(elapsed_chacha) + ' on average')
print ('Cipher text: ' + str(ciphertext))

print ('Calling KEY GENERATION encryption')
ciphertext, elapsed_keygen = key(text, iterations)
print ('Took ' + str(elapsed_keygen) + ' on average')
print ('Cipher text: ' + str(ciphertext))


# print ('Calling present encryption')
# ciphertext, elapsed_p = present(text, iterations)
# print ('Took ' + str(elapsed_p) + ' on average')
# print ('Cipher text: ' + str(ciphertext))

print ('Calling HB2 encryption')
ciphertext, elapsed_hb = hb2Encryption(plaintext, iterations)
print ('Took ' + str(elapsed_hb) + ' on average')
print ('Cipher text: ' + str(ciphertext))

print ('Calling xtea encryption')
ciphertext, elapsed_x = xsmth(text, iterations)
print ('Took ' + str(elapsed_x) + ' on average')
print ('Cipher text: ' + str(ciphertext))

text = b"{\"system\":{\"set_relay_state\":{\"state\":1}}}000000"
print ('Calling aes encryption')
ciphertext, elapsed_aes = aes(text, iterations)
print ('Took ' + str(elapsed_aes) + ' on average')
print ('Cipher text: ' + str(ciphertext))

print ('Extra time per iteration for hb = ' + str((elapsed_hb - elapsed_ak) / iterations))
print ('Extra time per iteration for ecdh = ' + str((elapsed_ecdh - elapsed_ak) / iterations))
print ('Extra time per iteration for xtea = ' + str((elapsed_x - elapsed_ak) / iterations))
print ('Extra time per iteration for aes = ' + str((elapsed_aes - elapsed_ak) / iterations))
print ('Extra time per iteration for chacha = ' + str((elapsed_chacha - elapsed_ak) / iterations))

Calling autokey encryption
Took 0.015625 on average
Cipher text: bytearray(b'{"system":{"set_relay_state":{"state":1}}}')
Calling ecdh encryption
Took 1.390625 on average
Cipher text: b'dXS*\\\xe5\xadqg\xfd\x8dL\xe71\rJ\x1b\t#+\xd7+\t\xda\xbd\xf0\x82\x8b\xb8^\xa3\x8b\xad\xbd\xccn\x17A\xed\x03\xd1+\xfd\xdb~\xe1\xed/'
Calling chacha encryption
Took 0.0 on average
Cipher text: b'\xf3jR9di\xf7\xb0j\xa8\x011X\xb1\x17^\xb4\x932t>\xd3B2\xfa2l\xce\xe7\x19P\x98\xcb\xac\xe4A\x9f\x03\x87\x06\x0e/'
Calling KEY GENERATION encryption
Took 1.28125 on average
Cipher text: b'\xa1\xde\x82L\x06Q;\x1b\xd0E\xea\x1cjm\x8fy}]J\xdb\x82R\x84a\xf2\xbb\x07VKE\xeam'
Calling HB2 encryption
Took 1.796875 on average
Cipher text: 11683
Calling xtea encryption
Took 0.234375 on average
Cipher text: b'\xc1\x88>\x19\r\xb2Or\xc23\x9c\tHVI\x9cQj\xe3\xbe\xa9\x11\xbc\xc4\xe7\xc8qG\xc5\xb1\xf1\xa6\xb2\x01\xcft\x99\xd8\\\x89\x86\xc9'
Calling aes encryption
Took 0.03125 on average
Cipher text: b'(\\\x93\xc0\xe4\x1b\xccv\xad\x8a